In [3]:
#relevant Libraries
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [11]:
#Data
#Data will be downloaded in C:\Users\sharm\tensorflow_datasets\
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

In [13]:
mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

### Splitting The data
By default the Tensorflow datasets don't have validation dataset, it just has train and test
So we need to divide the data accordingsly

In [18]:
#Here we have taken 10% that it 0.1 of train data
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64) #This will cast values to integers

In [20]:
#To store test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

### Function to Scale the inputs 
It will take MNIST image and it's label

In [37]:
'''Batch_size = 1 = Stochastic Gradient descent (SGD)
batch_size = #Samples = (single batch) GD
so, 1<batch size<# samples = mini-batch GD'''

def scale(image,label):
    image = tf.cast(image,tf.float32)   #typecasting to float
    image /= 255.
    return image, label
    
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000   #it's shuffling the batches 
shuffle_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffle_train_and_validation_data.take(num_validation_samples)
train_data = shuffle_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

### Outlining the Model

In [82]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),    
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choosing the optimizer and the Loss Function

In [85]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy']) 

### Training

In [88]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, 
          validation_data= (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 2s - 3ms/step - accuracy: 0.9048 - loss: 0.3290 - val_accuracy: 0.9547 - val_loss: 0.1524
Epoch 2/5
540/540 - 1s - 2ms/step - accuracy: 0.9607 - loss: 0.1314 - val_accuracy: 0.9630 - val_loss: 0.1199
Epoch 3/5
540/540 - 1s - 2ms/step - accuracy: 0.9704 - loss: 0.0950 - val_accuracy: 0.9743 - val_loss: 0.0840
Epoch 4/5
540/540 - 1s - 2ms/step - accuracy: 0.9779 - loss: 0.0706 - val_accuracy: 0.9780 - val_loss: 0.0684
Epoch 5/5
540/540 - 1s - 2ms/step - accuracy: 0.9819 - loss: 0.0581 - val_accuracy: 0.9833 - val_loss: 0.0580


### Test The Model

In [91]:
test_loss,test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.9732 - loss: 0.0874


In [95]:
print('Test Loss: {0:.2f}, Test Accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100.))

Test Loss: 0.09, Test Accuracy: 97.32%
